## This notebook can be used to create a fine-tuned model, using the OpenAI API. With gpt-4o as the teacher and gpt-4o-mini as the student.

### Steps to run:
1. In `utils.py`, set a path to an .env file containing an export OPENAI_API_KEY="".
2. Define a response model in the second code cell, see instructions above that cell.
3. Set constants in the third code cell to control prompts, models, and other parameters.
4. Change the pydantic name in the fourth code cell to the response model you defined.
5. Run the cells in sequence down to the 11th code cell, the 11th code cell will set up a finetuning job, which may take a few hours to complete. OpenAI will email you when it's done if you have the setting enabled.
6. The 12th code cell will compare the base mini model, the finetuned mini model, and the teacher model on the test set, using vector similarity to compare the outputs.

In [10]:
# Fine-tuning Pipeline Notebook

import sys
from pathlib import Path
from typing import List, Literal, Type, Any, Dict, Optional
from pydantic import BaseModel, Field

# Add the src directory to the Python path
sys.path.append(str(Path.cwd().parent))

from src.batch_preparation import prepare_batch_file
from src.batch_processing import upload_batch_file, create_batch_job, wait_for_batch_completion, process_batch_results
from src.data_processing import prepare_finetuning_data, validate_finetuning_data
from src.finetuning import prepare_and_start_finetuning
from src.evaluation import evaluate_models, monitor_finetuning_job


# Function to load prompts from a text file
def load_prompts_from_file(file_path: str) -> List[str]:
    with open(file_path, 'r') as f:
        return [line.strip() for line in f if line.strip()]

### Define your response model below using a pydantic BaseModel. This is an easy way to use pythonic concepts when describing to the LLM how to structure its response. Copy the class to use when deploying the finetuned model, passing it into the response_model parameter.

In [11]:
# Define your response model (example)
class SentimentAnalysis(BaseModel):
    sentiment: Literal["positive", "negative", "neutral"] = Field(description="Overall sentiment of the text")
    intensity: float = Field(description="Strength of the sentiment, from 0.0 to 1.0")
    label: str = Field(description="Single word category describing the main focus of the sentiment (e.g., 'service', 'food', 'price')")

### Running this cell takes the model you defined above and uses it to set the RESPONSE_MODEL variable, which is used in the rest of the notebook.

In [12]:
# Function to get the most recently defined Pydantic model
# the purpose of this is just to pass the above object to other functions
# in this notebook, to save on needing to pass the object as a parameter manually.

def get_latest_pydantic_model() -> Type[BaseModel]:
    models = [cls for name, cls in globals().items() if isinstance(cls, type) and issubclass(cls, BaseModel) and cls != BaseModel]
    if not models:
        raise ValueError("No Pydantic model defined. Please define a model before running the pipeline.")
    return models[-1]

RESPONSE_MODEL = get_latest_pydantic_model()
MODEL_NAME: str = RESPONSE_MODEL.__name__

### This is where you set the model parameters. Most importantly the system message for your use case.

In [13]:
# Set up file paths and model parameters
SYSTEM_MESSAGE: str = "You are a sentiment analysis model. You will be given a text and asked to analyze the sentiment of the text. You will return a sentiment, intensity, and label."
LARGE_MODEL: str = "gpt-4o-2024-08-06"
MINI_MODEL: str = "gpt-4o-mini-2024-07-18"
MAX_TOKENS: int = 2000
SUFFIX: str = f"{MODEL_NAME}_v1"  # Change this for different versions of your model


### Change the prompts file path to the path of your prompts file. Leave the other paths as they are, unless you need the intermediate outputs to go somewhere else.

In [14]:
PROMPTS_FILE_PATH: str = "../data/prompts/sentiment_analysis_prompts.txt"
BATCH_INPUT_DIR: Path = Path("../data/batch_files/batch_inputs")
BATCH_OUTPUT_DIR: Path = Path("../data/batch_files/batch_outputs")
FINETUNE_DIR: Path = Path("../data/finetune_files")

In [15]:
# Load prompts
prompts: List[str] = load_prompts_from_file(PROMPTS_FILE_PATH)
print(f"Loaded {len(prompts)} prompts from {PROMPTS_FILE_PATH}")


Loaded 50 prompts from ../data/prompts/sentiment_analysis_prompts.txt


In [16]:

# Step 1: Prepare batch file
batch_input_path = prepare_batch_file(
    prompts=prompts,
    response_model=RESPONSE_MODEL,
    system_message=SYSTEM_MESSAGE,
    model=LARGE_MODEL,
    max_tokens=MAX_TOKENS,
    save_dir=BATCH_INPUT_DIR,
    filename_prefix=MODEL_NAME
)


Batch input file created at: ../data/batch_files/batch_inputs/SentimentAnalysis_batch_input.jsonl


### In theory this cell can take up to 24h to run, depending on the business of the batch API. In practice, it's usually done in a few minutes.

In [17]:
# Step 2: Process batch
batch_file_id = upload_batch_file(batch_input_path)
batch_job_id = create_batch_job(batch_file_id)
completed_batch = wait_for_batch_completion(batch_job_id)

Batch job created with ID: batch_SBzD9kac7okiv3iP8QRPfbAz
Batch status: validating. Waiting 60 seconds...
Batch status: in_progress. Waiting 60 seconds...
Batch job batch_SBzD9kac7okiv3iP8QRPfbAz completed


In [18]:
# Step 3: Process batch results
batch_output_path = process_batch_results(
    batch_job_id,
    BATCH_OUTPUT_DIR,
    f"{MODEL_NAME}_batch_output"
)
print(f"Batch output saved to: {batch_output_path}")

Batch results saved to: ../data/batch_files/batch_outputs/SentimentAnalysis_batch_output_batch_SBzD9kac7okiv3iP8QRPfbAz.jsonl
Batch output saved to: ../data/batch_files/batch_outputs/SentimentAnalysis_batch_output_batch_SBzD9kac7okiv3iP8QRPfbAz.jsonl


In [19]:
# Step 4: Prepare fine-tuning data
train_file, test_file = prepare_finetuning_data(
    batch_input_path=batch_input_path,
    batch_output_path=batch_output_path,
    output_dir=FINETUNE_DIR,
    output_filename_prefix=MODEL_NAME
)
print(f"Training file created: {train_file}")
print(f"Testing file created: {test_file}")

if validate_finetuning_data(train_file) and validate_finetuning_data(test_file):
    print("Fine-tuning data is valid.")
else:
    raise ValueError("Fine-tuning data is invalid.")

Training data saved to: ../data/finetune_files/train/SentimentAnalysis_train.jsonl
Testing data saved to: ../data/finetune_files/test/SentimentAnalysis_test.jsonl
Training file created: ../data/finetune_files/train/SentimentAnalysis_train.jsonl
Testing file created: ../data/finetune_files/test/SentimentAnalysis_test.jsonl
Fine-tuning data is valid.


In [20]:
# Step 5: Start fine-tuning
job_id = prepare_and_start_finetuning(
    training_file_path=train_file,
    validation_file_path=test_file,
    model=MINI_MODEL,
    suffix=SUFFIX
)

print(f"\nFine-tuning job started with ID: {job_id}")

Fine-tuning file uploaded with ID: file-5OK1M9wo7nM9mnxdWX4AACaV
Fine-tuning file uploaded with ID: file-qxwO65nGQKV0DK9AYymPbkJf
Fine-tuning job created with ID: ftjob-5955i0ExbrHMNvh8ovrokdDL

Fine-tuning job started with ID: ftjob-5955i0ExbrHMNvh8ovrokdDL


In [21]:
finetuned_model_name = monitor_finetuning_job(job_id)
print(f"Fine-tuned model name: {finetuned_model_name}")

Fine-tuning job status: validating_files. Waiting 60 seconds...
Fine-tuning job status: running. Waiting 60 seconds...
Fine-tuning job status: running. Waiting 60 seconds...
Fine-tuning job status: running. Waiting 60 seconds...
Fine-tuning job status: running. Waiting 60 seconds...
Fine-tuning job status: running. Waiting 60 seconds...
Fine-tuning job status: running. Waiting 60 seconds...
Fine-tuning job status: running. Waiting 60 seconds...
Fine-tuning job status: running. Waiting 60 seconds...
Fine-tuning job status: running. Waiting 60 seconds...
Fine-tuning job status: running. Waiting 60 seconds...
Fine-tuning job status: running. Waiting 60 seconds...
Fine-tuning job ftjob-5955i0ExbrHMNvh8ovrokdDL completed successfully.
Fine-tuned model name: ft:gpt-4o-mini-2024-07-18:pathlabs:sentimentanalysis-v1:9wdfmppr


### Wait for the finetune job to complete before running the next cell. The next cell will compare the vector similarity of the responses of three models: the base mini model, the finetuned mini model, and the teacher model.

If the environment variable for finetuned_model_name is lost through the course of the notebook, you can manually set it in the next cell. If this happens rerun the first 6 code cells before running the next 2 cells.

In [22]:

evaluation_results = evaluate_models(
    validation_file=test_file,
    finetuned_model=finetuned_model_name,
    base_mini_model=MINI_MODEL,
    large_model=LARGE_MODEL,
    max_tokens=MAX_TOKENS,
    save_dir=Path("../data/evaluation_results"),
    response_model=RESPONSE_MODEL
)

print("Evaluation results:")
for model, result_path in evaluation_results["results"].items():
    print(f"{model}: {result_path}")

print("\nModel similarities:")
for comparison, similarity in evaluation_results["similarities"].items():
    print(f"{comparison}: {similarity:.4f}")

Batch job created with ID: batch_dDMpUs1XoHB8J5GbzdWQBH6V
Batch status: validating. Waiting 60 seconds...


Exception: Batch job batch_dDMpUs1XoHB8J5GbzdWQBH6V failed